# Contrast calculation comparing three methods

Comparing E2E DH contrast vs. contrast from PASTIS image DH vs. PASTIS matrix contrast; following script `contrast_calculation_simple.py` but with less ouputs.

In [ ]:
import os
import numpy as np
from astropy.io import fits
import astropy.units as u
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

os.chdir('../../pastis/')
from config import CONFIG_PASTIS
import util as util
import image_pastis as impastis
import webbpsf_imaging as webbim

In [ ]:
# Reading parameters from configfile
dataDir = os.path.join(CONFIG_PASTIS.get('local', 'local_data_path'), 'active')
which_tel = CONFIG_PASTIS.get('telescope', 'name')
nb_seg = CONFIG_PASTIS.getint(which_tel, 'nb_subapertures')
filter = CONFIG_PASTIS.get(which_tel, 'filter_name')
fpm = CONFIG_PASTIS.get(which_tel, 'focal_plane_mask')         # focal plane mask
lyot_stop = CONFIG_PASTIS.get(which_tel, 'pupil_plane_stop')   # Lyot stop
inner_wa = CONFIG_PASTIS.getint(which_tel, 'IWA')
outer_wa = CONFIG_PASTIS.getint(which_tel, 'OWA')
sampling = CONFIG_PASTIS.getfloat(which_tel, 'sampling')
zern_number = CONFIG_PASTIS.getint('calibration', 'zernike')
zern_mode = util.ZernikeMode(zern_number)
zern_max = CONFIG_PASTIS.getint('zernikes', 'max_zern')
wvln = CONFIG_PASTIS.getfloat(which_tel, 'lambda') * u.nm

In [ ]:
# Define matrix mode (numerical or analytical)
matrix_mode = 'analytical'
# Define RMS value to work with
rms = 100 * u.nm

In [ ]:
# Import PASTIS matrix
if matrix_mode == 'numerical':
    filename = 'PASTISmatrix_num_' + zern_mode.name + '_' + zern_mode.convention + str(zern_mode.index)
    matrix_pastis = fits.getdata(os.path.join(dataDir, 'matrix_numerical', filename + '.fits'))
elif matrix_mode == 'analytical':
    filename = 'PASTISmatrix_' + zern_mode.name + '_' + zern_mode.convention + str(zern_mode.index)
    matrix_pastis = fits.getdata(os.path.join(dataDir, 'matrix_analytical', filename + '.fits'))

In [ ]:
# Display the matrix
plt.figure(figsize=(10, 10))
plt.imshow(matrix_pastis)

In [ ]:
# Load baseline contrast
contrastname = 'base-contrast_' + zern_mode.name + '_' + zern_mode.convention + str(zern_mode.index)
contrast_base = float(np.loadtxt(os.path.join(dataDir, 'calibration', contrastname+'.txt')))
print("Base contrast: {:.2e}".format(contrast_base))

### Generate random aberraiton coefficients normalized to rms value

In [ ]:
# Create random aberration coefficients - piston only
Aber = np.random.random([nb_seg])     # piston values in input units

# Remove global piston
Aber -= np.mean(Aber)

print('PISTON ABERRATIONS:', Aber)

In [ ]:
# Normalize to the RMS value I want
rms_init = util.rms(Aber)
print("Initial rms: {} nm".format(rms_init))
coef = Aber * rms.value / rms_init
calc_rms = util.rms(coef) * u.nm
coef *= u.nm    # making sure the aberration has the correct units
print("Calculated RMS:", calc_rms)

# Modulo wavelength to get rid of phase wrapping
for i, k in enumerate(coef):
    if k < 0:
        coef[i] = -(np.abs(coef[i]) % wvln)
    else:
        coef[i] = coef[i] % wvln
print(coef)
print("Wavelength: {}".format(wvln))

In [ ]:
# Make equivalent aberration array that goes into the WebbPSF function
Aber_WSS = np.zeros([nb_seg, zern_max])
Aber_WSS[:,0] = coef.to(u.m).value   # index "0" works because we're using piston currently; convert to meters

print("WebbPSF aberrations in meters:\n", Aber_WSS)

## Baseline direct PSF contrast

In [ ]:
# Generating baseline PSF from WebbPSF - no coronagraph, no aberrations - and normalize it
psf_perfect = webbim.nircam_nocoro(filter, np.zeros_like(Aber_WSS))
normp = np.max(psf_perfect)
psf_perfect = psf_perfect / normp

plt.figure(figsize=(10, 10))
plt.imshow(psf_perfect, norm=LogNorm())

## Contrast calculations

### WebbPSF coro

In [ ]:
psf_webbpsf = webbim.nircam_coro(filter, fpm, lyot_stop, Aber_WSS)
psf_webbpsf = psf_webbpsf / normp

plt.figure(figsize=(10, 10))
plt.imshow(psf_webbpsf, norm=LogNorm())
plt.title("WebbPSF coro PSF")

In [ ]:
# Take the DH
dh_area = util.create_dark_hole(psf_webbpsf, inner_wa, outer_wa, sampling)
webb_dh_psf = psf_webbpsf * dh_area

dh_box = (outer_wa + 3)* sampling

plt.figure(figsize=(10, 10))
plt.imshow(util.zoom_cen(webb_dh_psf, dh_box), norm=LogNorm())
plt.colorbar()
plt.title("WebbPSF coro DH")

In [ ]:
# Calculate WebbPSF DH contrast
contrast_webbpsf = np.mean(webb_dh_psf[np.where(webb_dh_psf != 0)])
print("WebbPSF contrast: {:.2e}".format(contrast_webbpsf))

### PASTIS image

In [ ]:
# Generate analytical image with PASTIS
psf_am, full_psf = impastis.analytical_model(zern_number, coef, cali=True)

# Display full PSF
plt.figure(figsize=(10, 10))
plt.imshow(util.zoom_cen(full_psf, psf_webbpsf.shape[0]/2), norm=LogNorm())
plt.title("PSF image PSF")

In [ ]:
# Display PASTIS image DH
plt.figure(figsize=(10, 10))
plt.imshow(util.zoom_cen(psf_am, dh_box), norm=LogNorm())
plt.colorbar()
plt.title("PASTIS image DH")

In [ ]:
# Calculate PASTIS image DH contrast
contrast_am = np.mean(psf_am[np.where(psf_am != 0)]) + contrast_base
print("PASTIS image constrast: {:.2e}".format(contrast_am))

### PASTIS matrix

In [ ]:
# Calculate contrast with PASTIS matrix
contrast_matrix = util.pastis_contrast(coef, matrix_pastis) + contrast_base
print("PASTIS matrix constrast: {:.2e}".format(contrast_matrix))

## Final display

In [ ]:
plt.figure(figsize=(18, 9))
plt.subplot(1, 2, 1)
plt.imshow(util.zoom_cen(webb_dh_psf, dh_box), norm=LogNorm())
plt.colorbar()
plt.title("WebPSF DH")
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(psf_am, dh_box), norm=LogNorm())
plt.colorbar()
plt.title("PASTIS DH")

print("CONTRASTS:")
print("WebbPSF: {:.2e}".format(contrast_webbpsf))
print("PASTIS image: {:.2e}".format(contrast_am))
print("PASTIS matrix: {:.2e}".format(contrast_matrix))